# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


In [ ]:
# We import all the necessary libraries and tools
!pip install selenium
!pip install webdriver-manager
import time
import shutil
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_info(index):
    # Configure Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.binary_location = 'C:/Program Files/Google/Chrome/Application/chrome.exe'

    # Get the driver path using ChromeDriverManager
    driver_path = ChromeDriverManager().install()

    # Create a webdriver instance with ChromeOptions
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()

    url = 'https://infogob.jne.gob.pe/Eleccion'
    driver.get(url)

    # Select elections button
    elections_button_selector = '#section > div.content-busqueda > div.cajas-busqueda.fondo-grisclaro > div.form-group.row > div:nth-child(1) > div > div.text'
    elections_button = driver.find_element(By.CSS_SELECTOR, elections_button_selector)
    elections_button.click()

    time.sleep(3)

    # Select presidential elections option
    presidential_elections_xpath = '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]'
    presidential_elections_option = driver.find_element(By.XPATH, presidential_elections_xpath)
    presidential_elections_option.click()

    
    time.sleep(5)
    
    # The code clicks an elections button and captures associated year elements

    elecciones_button_selector2 = '#section > div.content-busqueda > div.cajas-busqueda.fondo-grisclaro > div.form-group.row > div:nth-child(2) > div'
    elecciones_button2 = driver.find_element(By.CSS_SELECTOR, elecciones_button_selector2)
    elecciones_button2.click()
    elecciones_años= elecciones_button2.find_elements(By.CLASS_NAME, "item")

    time.sleep(5)
    
    # In this part, the provided code performs web interactions, including clicking on elements and navigating through pages
    elecciones_años[index].click()

    driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button/span").click()
    driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a").click()

    time.sleep(3)
    html = driver.page_source

    # We select the election name, total votes, and political party name from the parsed HTML, creating a DataFrame with the extracted data, and then quit the driver
    soup = BeautifulSoup(html, 'html.parser')

    nombre_eleccion_element = soup.find('h3', {'class': 'color-negro font-exo300'})
    nombre_eleccion = nombre_eleccion_element.text.strip() if nombre_eleccion_element else None

    monto_votos_totales_selector = '#gridEleccionResultadosElectorales > tbody > tr:nth-child(1) > td:nth-child(4)'
    monto_votos_totales_element = soup.select_one(monto_votos_totales_selector)
    monto_votos_totales = monto_votos_totales_element.text.strip() if monto_votos_totales_element else None

    partido_politico_element = soup.find('a', {'class': 'link color-negro alias'})
    partido_politico_nombre = partido_politico_element.text.strip() if partido_politico_element else None

    data = [[nombre_eleccion, partido_politico_nombre, monto_votos_totales]]
    columnas = ['Elecciones', 'ORGANIZACIÓN POLÍTICA', 'VOTOS TOTALES']
    df = pd.DataFrame(data, columns=columnas)

    driver.quit()

    return df

In [ ]:
dfs = []

# Iterate over the indices and store the DataFrames in the list
for i in range(1, 26):
    df = scrape_info(i)
    dfs.append(df)

# Concatenate all the DataFrames into one
result_df = pd.concat(dfs, ignore_index=True)

# Display the resulting DataFrame
result_df


In [ ]:
result_df.to_excel('group_3_ass_5_2024.xlsx', index=False)
